# Setup

Importing all neccessary libraries onto the drive:


*   Standard Python Libraries
*   TensorFlow and Keras
*   Libraries for GradCAM implementation





Firstly, the necessary libraries must imported and loaded by the Colab instance. Those of note are the NumPy, Pandas and matplotlib libraries as they will be used to manipulate the dataset and prepare it for pre-processing.

Additionally, various parts and components of the TensorFlow and Keras libraries are imported too. Some are imported directly for convenience to avoid writing out long lines of code to access them.

Lastly, the final snippet of code allows for the automatic mounting of the Google Drive to this Jupyter Notebook file and, therefore, access the echocardiogram dataset stored. It is worth noting that because of the way Drive works, the original user must be running the code in order for the notebook to be able to access the drive stored on their account.

In [1]:
#Import: General
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pathlib
import seaborn as sn
import sklearn.metrics

#Import: ResNet-50
from keras.preprocessing import image
from keras.applications import ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
import keras

#Import: Grad-CAM
from tensorflow.keras.models import Model
from google.colab.patches import cv2_imshow
import cv2
import imutils

#Mount Drive
#from google.colab import drive
#drive.mount("drive")

# Grad-CAM

Grad-CAM allows to see what the CNN 'sees'. Great for analysing and understanding where the neural nodes activate and if the model works as intended.

In [ ]:
class GradCAM:

  def __init__(self, model, classIdx, layerName=None):
    self.model = model
    self.classIdx = classIdx
    self.layerName = layerName

    if self.layerName is None:
      self.layerName = self.find_target_layer()

  def find_target_layer(self):
    for layer in reversed(self.model.layers): #loops through all layers from the last
      if len(layer.output_shape) == 4: #until it finds a layer with an output shape of 4 as required
        print(layer) #otherwise the Grad-CAM will not work as a heatmap cannot be built
        return layer.name
    raise ValueError("Could not find a 4D layer. GradCAM cannot work.") #for incompatiable models

  def compute_heatmap(self, image, eps=1e-8): #constructs the heatmap
    gradModel = Model ( #creating a blank model object and assinging the CNN model to it


        inputs = [self.model.inputs],
        outputs = [self.model.get_layer(self.layerName).output, self.model.output]
    )

    with tf.GradientTape() as Tape: #creates the gradient tape object
      inputs = tf.cast(image, tf.float32)
      (convOutputs, predictions) = gradModel(inputs) #runs the prediction and grabs the results
      loss = predictions[:, self.classIdx] #grabs the loss value from the results
      grads = Tape.gradient(loss, convOutputs) #finds the gradient values

    castConvOutputs = tf.cast(convOutputs > 0, "float32")
    castGrads = tf.cast(grads > 0, "float32")
    guidedGrads = castConvOutputs * castGrads * grads #calculates the guided gradient values

    convOutputs = convOutputs[0]
    guidedGrads = guidedGrads[0]

    weights = tf.reduce_mean(guidedGrads, axis=(0, 1)) #calculates weight values
    cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1) #finds the final values to be used by the heatmap

    (w, h) = (image.shape[2], image.shape[1]) #begins drawing out the heatmap
    heatmap = cv2.resize(cam.numpy(), (w, h)) #resizing to fit the input image dimensions
    numer = heatmap - np.min(heatmap)
    denom = (heatmap.max() - heatmap.min()) + eps
    heatmap = numer / denom
    heatmap = (heatmap * 255).astype("uint8")

    return heatmap #returns the heatmap as an output of this function

  def overlay_heatmap(self, heatmap, image, alpha=0.5, colourmap=cv2.COLORMAP_VIRIDIS): #Overlays the heatmap onto the image
    heatmap = cv2.applyColorMap(heatmap, colourmap)
    output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

    return heatmap, output #returns both the heatmap and the final output image

# Building the Database

In [ ]:
#Defining dataset parameters
batch_size = 32 #Size of batches for the data
img_height = 224 #Height of the images
img_width = 224 #Width of the images

#Grabbing the echocardiogram folder
dataset_url = "/content/drive/MyDrive/Project/dataset/echocardiograms" #google drive of the database
data_dir = pathlib.Path(dataset_url).with_suffix('') #sets up the directory to be used
classnames = ['a4c', 'a2c'] #Array to assign labels to the classes [1, 0]

#Training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir, #the directory being converted
  validation_split=0.2, #20% used for validation, 80% for actual training
  subset="training", #this is the training subset being outputted
  seed=132, #seed to randomise data order and seperation
  image_size=(img_height, img_width), #converts image sizes to required
  batch_size=batch_size, #the batch size of the dataset
  class_names=classnames, #assinging labels to the classes
  label_mode='int' #controls the datatype of the class labels i.e. [1, 0]
  )

#Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory( #same as above but for the validation dataset
  data_dir,
  validation_split=0.2,
  subset="validation", #for validation subset
  seed=132,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  class_names=classnames,
  label_mode='int'
  )

#Manually records the class labels for each echocardiogram in the validation dataset for later use
labels =  np.array([])
labels = np.concatenate([y for x, y in val_ds], axis=0)

#Dataset pre-processing
preprocess_input = keras.applications.resnet.preprocess_input #data pre-processing function for ResNet-50

#Running the echocardiogram images through a pre-processing neural network layer and saving the new results
train_ds = train_ds.map(lambda images, labels:
                        (preprocess_input(images), labels))
val_ds = val_ds.map(lambda images, labels:
                    (preprocess_input(images), labels))

# ResNet-50

Instantiating a ResNet50 model

In [ ]:
#Pre-trained ResNet-50 v2 model
resnet50 = ResNet50(
    include_top=False, #Does not include the top layers used for classification
    weights='imagenet', #pre-trained using the imagenet dataset
    input_shape=(img_height,img_width,3), #size of the input images
    pooling=None, #no additional pooling layers
    classes=1000 #irrelevant here as no top layers included
)

Applying transfer learning

In [ ]:
#Freezing the resnet model
resnet50.trainable = False #sets resnet50 to be untrainable

#Sequentional New Model
model = keras.Sequential() #keras structure to create a CNN model by sequentially adding layers
model.add(resnet50) #adds resnet50 as the first 'layer'
model.add(keras.layers.Identity()) #blank layer used by Grad-CAM to peer into the model. The layer does nothing.
model.add(keras.layers.GlobalAveragePooling2D()) #Pooling layer that takes a global average of its input and outputs the answer.
model.add(keras.layers.Dropout(0.2)) #Dropout layer is used to reduce overfitting in models by 'dropping out' by setting some of the input units to 0 at a rate of its function input (0.2).
model.add(keras.layers.Dense(2, activation='softmax')) #A fully-connected (Dense) layer with a softmax activiation function to output an array of probabilities for the viewpoints.

#Model Summary
model.summary(show_trainable=True) #summary of the final model and its layers

# Training

Training and validating the CNN model. As only the top layers are set to be trainable, only they will be trained. However, the entire model is validated.


In [ ]:
#Setup
callback_list = [keras.callbacks.CSVLogger('/content/drive/MyDrive/Project/results/epoch_log.csv', separator=",", append=False), #will log results per epoch into a csv file
                 keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Project/results/tensor_logs')] #will take logs of the tensors for debugging


#Compiling the model
model.compile(
  optimizer='adamax', #using adamax optimizer
  loss=tf.keras.losses.SparseCategoricalCrossentropy(), #using sparce categorical crossentropy loss function
  metrics=[keras.metrics.SparseCategoricalAccuracy(), #using accuracy (multi-class model version) and poisson metrics
           keras.metrics.Poisson()]
)

#Training the model
score = model.fit(
  train_ds, #training dataset
  validation_data=val_ds, #validation dataset
  epochs=20, #number of training steps i.e. number of times the model is trained using the training dataset
  callbacks=callback_list, #runs additional functions in between training
  use_multiprocessing=True #Uses multi-processing for better computational power usage
).history #Recording the metrics into the array 'score'

# Model Cache

This allows for the saving and loading (i.e. storage) of the CNN model. As a result, various versions of the model can be trained, saved and then loaded as needed.

Model versions (and file name):

*   Final CNN model (Model)
*   CNN model with 256x256 resolution (Model_256)




In [ ]:
#Saving the Model
model.save("/content/drive/MyDrive/Project/code/Model")
model.summary(show_trainable=True)

In [ ]:
#Load Saved Model
model = keras.models.load_model("/content/drive/MyDrive/Project/code/Model")
model.summary(show_trainable=True)

# Results

Uses the metrics obtained from the training section to plot out graphs for them against each training step.

In [ ]:
#Plotting Loss per epoch
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,1])
line1 = plt.plot(score["loss"], label='Training')
line2 = plt.plot(score["val_loss"], label='Validation')
plt.legend()

#Plotting Accuracy per epoch
plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(score["sparse_categorical_accuracy"], label='Training')
plt.plot(score["val_sparse_categorical_accuracy"], label='Validation')
plt.legend()

#Plotting Poisson per epoch
plt.figure()
plt.ylabel("Poisson")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(score["poisson"], label='Training')
plt.plot(score["val_poisson"], label='Validation')
plt.legend()

# Testing

This section is used for manually testing and debugging the model. Images from a testset were used to analyse the inner-workings of the CNN model to better understand it.

In [ ]:
#Preparing the test image
url = "/content/drive/MyDrive/Project/dataset/testset/ES00058 N_4CH_2_002.jpg" #Taking a test image by its url
orig = cv2.imread(url)
resized = cv2.resize(orig, (img_width, img_height)) #resizing it

image = load_img(url, target_size=(img_width,img_height)) #loading the image
image = img_to_array(image) #converting image into an array for pre-processing and classification
image = np.expand_dims(image, axis=0)
image = train_ds = keras.applications.resnet.preprocess_input(image) #pre-processing the image

In [ ]:
#Prediction
preds = model.predict(image) #classifying the image
i = np.argmax(preds[0]) #Grabbing the classification from the output array
label = classnames[np.argmax(preds[0])] + ": " + str(max(preds[0])) #Translating it to be human-readable

print(preds[0]) #Prints array of predictions
print("The predicted class is", classnames[np.argmax(preds[0])]) #Prints the human-readable class label

In [ ]:
#Grad CAM
cam = GradCAM(model, i) #Created grad-cam object
heatmap = cam.compute_heatmap(image) #calculates the heatmap for the test image
heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0])) #resizes the heatmap to match image dimensions
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5) #overlays the heatmap onto the image

cv2.rectangle(output, (0, 0), (340, 40), (0, 0, 0), -1) #adding a label indicating the softmax prediction probability value
cv2.putText(output, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

output = np.vstack([orig, heatmap, output]) #creating a stack of images
output = imutils.resize(output, height=700) #resizing the stack
cv2_imshow(output) #displaying the results
cv2.waitKey(0)

## Confusion Matrix

In [ ]:
y_prediction = model.predict(val_ds) #manually performing a validation check again
y_pred = np.argmax(y_prediction, axis=1) #grabbing the prediction values

In [ ]:
result = tf.math.confusion_matrix(labels, y_pred, num_classes=2) #calculates the confusion matrix
df_cm = pd.DataFrame(result, range(2), range(2)) #converts it into the Pandas Dataframe object
sn.set(font_scale=1.4) #Sets label size
hm = sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) #Creates a heatmap-styled figure
plt.show() #displays result

In [ ]:
#calculates precision and recall for double-checking the manual calculations
matrix = sklearn.metrics.classification_report(labels, y_pred, target_names=classnames, output_dict=False)
print(matrix)

# Model Infomation

This code can be used to create a block diagram of the final CNN model and the ResNet-50 model used. The block diagrams show every layer as well as relvant information for each layer.

In [ ]:
#Model Information
keras.utils.plot_model(model, to_file='/content/drive/MyDrive/Project/results/model.png', show_shapes=True)
keras.utils.plot_model(resnet50, to_file='/content/drive/MyDrive/Project/results/resnet50.png', show_shapes=True)